In [1]:
from controllerLib import *
robot, scene, group, zeroPose  = initRobot()

Manipulator detected
World detetected
Initial Join values should be [77.32632695, -69.73220526, 121.29976102, -141.59751593, -90.06095192, 168.36293361]
Loading points
Total points loaded:  36
Setting Scene
Definning zeroPose


In [3]:
goToZero()

Adding Constraints


In [3]:
pushTwo((2, 7), (2, 10))

Total points loaded:  36
Total time 4.40146899223


In [4]:
pushAll()

Error


In [4]:
pushOne(2, 1)

Error


In [3]:
createPath([(0, 0), (2, 4), (2, 1), (2, 9), (0, 0), (2, 8), (2, 11), (2, 7), (0, 3)])

Total points loaded:  36
Length 9
Total time 28.9839551449


In [4]:
createPath([[0,0],[2,11],[2,0],[0,11]])

Total points loaded:  36
Length 4
Total time 17.9420590401


In [5]:
robot